In [1]:
# gain#2 Lin. FI/FO
import ROOT as r
import numpy as np
%jsroot

r.gROOT.ProcessLine(".x ../../rootlogon.C")

file_path = '../../ROOT/f3ic_test0173.root'
root_file = r.TFile(file_path)

NumOfHist = 7

add include path : /home/exp/ANAROOT/anaroot/include


In [2]:
r.tree.Draw("F3IC_Eraw[0]>>h0(4e3,100,8.2e3)")
r.tree.Draw("F3IC_Eraw[1]>>h1(4e3,100,8.2e3)")
r.tree.Draw("F3IC_Eraw[2]>>h2(4e3,100,8.2e3)")
r.tree.Draw("F3IC_Eraw[3]>>h3(4e3,100,8.2e3)")
r.tree.Draw("F3IC_Eraw[4]>>h4(4e3,100,8.2e3)")
r.tree.Draw("F3IC_Eraw[5]>>h5(4e3,100,8.2e3)")

r.tree.SetAlias("F3IC_Eraw_sum", "F3IC_Eraw[0]+F3IC_Eraw[1]+F3IC_Eraw[2]+F3IC_Eraw[3]+F3IC_Eraw[4]+F3IC_Eraw[5]")
r.tree.Draw("F3IC_Eraw_sum>>h6(12e3, 600, 48e3)")

r.zone(3, 3)
r.ht(1,7)

r.c1.Draw()


 Draw ID:1
 Draw ID:2
 Draw ID:3
 Draw ID:4
 Draw ID:5
 Draw ID:6
 Draw ID:7


Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [3]:
r.ls()


 ---- List of Tree ----

        ID     Class                 Name                               Title
         1     TTree                 tree                                tree


 ---- List of Hist ----

        ID     Class                 Name                               Title
         1      TH1F                   h0                        F3IC_Eraw[0]
         2      TH1F                   h1                        F3IC_Eraw[1]
         3      TH1F                   h2                        F3IC_Eraw[2]
         4      TH1F                   h3                        F3IC_Eraw[3]
         5      TH1F                   h4                        F3IC_Eraw[4]
         6      TH1F                   h5                        F3IC_Eraw[5]
  -->    7      TH1F                   h6                       F3IC_Eraw_sum



In [3]:
def find_peaks(hist, threshold=10, min_distance=60):
    spec = r.TSpectrum()
    npeaks = spec.Search(hist, 2, "", threshold)
    
    peaks = []
    fits = []
    
    for i in range(npeaks):
        x = spec.GetPositionX()[i]
        bin = hist.FindBin(x)
        sigma = 40
        
        # 前のピークと距離が100以上離れているかチェック
        if len(peaks) > 0 and abs(x - peaks[-1][0]) < min_distance:
            print(f"Peak at {x} is too close to {peaks[-1][0]}, skipping.")
            continue
        
        fit_range = 1.2 * sigma
        
        fit = r.TF1(f"fit_{i}", "gaus", x - fit_range, x + fit_range)
        hist.Fit(fit, "RQ0")
        
        peak_x = fit.GetParameter(1)
        peak_sigma = fit.GetParameter(2)
        
        peaks.append((peak_x, peak_sigma))
        fits.append(fit)
    
    peaks.sort(key=lambda peak: peak[0])
    
    # min_distance より近いピークを削除する処理
    filtered_peaks = []
    for i in range(len(peaks)):
        if i == 0:  # 最初のピークは常に追加
            filtered_peaks.append(peaks[i])
        else:
            # 現在のピークと最後に追加したピークの距離を確認
            if abs(peaks[i][0] - filtered_peaks[-1][0]) >= min_distance:
                filtered_peaks.append(peaks[i])
            else:
                # 近い場合は小さい方を消す
                print(f"Removing peak at {peaks[i][0]} because it's too close to {filtered_peaks[-1][0]}")
    
    peaks = filtered_peaks

    #print("Final peaks:", peaks)
    
    return peaks, fits

expected_x = list(range(50, 191, 20)) + list(range(250,2501,50))

# Initialize canvases for each type of graph (3x2 grid)
cPeak = r.TCanvas("cPeak", "Peak Graphs", 1200, 1200)
cPeak.Divide(3, 3)

cRes = r.TCanvas("cRes", "Residual Graphs", 1200, 1200)
cRes.Divide(3, 3)

cPer = r.TCanvas("cPer", "Percentage Error Graphs", 1200, 1200)
cPer.Divide(3, 3)

cHwF = r.TCanvas("cHwF", "Hist with Gauss Fits", 1200, 1200)
cHwF.Divide(3, 3)

cHwF2 = r.TCanvas("cHwF2", "Hist with Gauss Fits2", 1200, 1200)
cHwF2.Divide(3, 3)

cHwF3 = r.TCanvas("cHwF3", "Hist with Gauss Fits3", 1200, 1200)
cHwF3.Divide(3, 3)

graphs = []
percentage_error_graphs = []
residual_graphs = []
fit_results = []
hists_with_fits = []

for i in range(NumOfHist):
    hist = r.gDirectory.Get(f"h{i}")
    peaks, fits = find_peaks(hist)
    
    x_values = []
    y_values = []
    y_errors = []
    
    for j, (x, sigma) in enumerate(peaks):
        if j < len(expected_x):
            x_values.append(expected_x[j])
            y_values.append(x)
            y_errors.append(sigma)
    
    n = len(x_values)
    graph = r.TGraphErrors(n, 
                           np.array(x_values, dtype=float), 
                           np.array(y_values, dtype=float), 
                           np.zeros(n), 
                           np.array(y_errors, dtype=float))
    
    graph.SetTitle(f"Peak Analysis for h{i}")
    graph.GetXaxis().SetTitle("Test input (mV)")
    graph.SetMinimum(0)
    graph.GetYaxis().SetTitle("MADC [ch]")
    graph.SetMarkerStyle(20)
    graph.SetMarkerColor(r.kBlue)
    graph.SetLineColor(r.kRed)
    
    cPeak.cd(i+1)
    graph.Draw("AP")

    # ここで線形フィット関数を作成
    linear_fit = r.TF1(f"linear_fit_h{i}", "pol1")  # 線形（一次）フィットを意味する "pol1"

    # データにフィット
    graph.Fit(linear_fit, "L", "", 0, 1200)  # "RQ"オプションはフィットメッセージを抑制

    # フィットされた関数を同じキャンバスに描画
    linear_fit.SetLineColor(r.kGreen)
    linear_fit.Draw("same")

    # フィット結果の傾きと切片を取得
    slope = linear_fit.GetParameter(1)  # 1番目のパラメータが傾き
    intercept = linear_fit.GetParameter(0)  # 0番目のパラメータが切片

    # 結果を表示
    print(f"切片: {intercept}, 傾き: {slope}")

    residuals = []
    for j in range(n):
        if x_values[j] <= max(x_values) and x_values[j] > 0:
            fitted_y = linear_fit.Eval(x_values[j])
            residual = y_values[j] - fitted_y
            residual /= max(y_values)
            residuals.append(residual)

    percentage_errors = []
    for j in range(n):
        percentage_error = (y_errors[j] / y_values[j]) * 100 if y_values[j] != 0 else 0
        percentage_errors.append(percentage_error)
    
    percentage_error_graph = r.TGraph(n, 
                                      np.array(x_values, dtype=float), 
                                      np.array(percentage_errors, dtype=float))
    percentage_error_graph.SetTitle(f"ノイズレベルの割合 for h{i}")
    percentage_error_graph.GetXaxis().SetTitle("test input [mV]")
    percentage_error_graph.GetYaxis().SetTitle("Percentage Error (%)")
    percentage_error_graph.SetMinimum(0)
    percentage_error_graph.SetMaximum(5)
    percentage_error_graph.SetMarkerStyle(22)
    percentage_error_graph.SetMarkerColor(r.kRed)

    # 新たに追加する3つの点
    x_additional = np.array([844, 270, 71], dtype=float)
    y_additional = np.array([1.0, 1.0, 1.7], dtype=float)
        
    # 3つの点を描画するためのTGraph
    additional_graph = r.TGraph(3, x_additional, y_additional)

    # 追加の点を元のグラフに重ねて描画
    additional_graph.SetMarkerStyle(21)  # マーカーのスタイルを設定
    additional_graph.SetMarkerColor(r.kBlue)  # マーカーの色を設定（赤色）
    
    cPer.cd(i+1)  
    percentage_error_graph.Draw("")
    
    # 追加の点を元のグラフに重ねて描画
    additional_graph.Draw("P same")  # "P"オプションで点を描画

    cPer.Update()
    percentage_error_graphs.append(percentage_error_graph)

    if residuals:
        residual_graph = r.TGraph(len(residuals), 
                                   np.array(y_values[:len(residuals)], dtype=float), 
                                   np.array(residuals, dtype=float))
        residual_graph.SetTitle(f"Residuals for h{i}")
        residual_graph.GetXaxis().SetTitle("MADC [ch]")
        residual_graph.GetYaxis().SetTitle("Residuals")
        residual_graph.SetMinimum(-5e-3)
        residual_graph.SetMaximum(+5e-3)
        residual_graph.SetMarkerStyle(21)
        residual_graph.SetMarkerColor(r.kRed)
        
        cRes.cd(i+1)
        residual_graph.Draw("ALP")
        residual_graphs.append(residual_graph)

    cHwF.cd(i+1)
    hist.Draw("")  
    
    for fit in fits:
        fit.SetLineColor(r.kRed)
        fit.Draw("same")

    cHwF2.cd(i+1)
    hist.Draw("")  
    
    for fit in fits:
        fit.SetLineColor(r.kRed)
        fit.Draw("same")

    cHwF3.cd(i+1)
    hist.Draw("")  
    
    for fit in fits:
        fit.SetLineColor(r.kRed)
        fit.Draw("same")
    
    hists_with_fits.append(hist)  # そのまま hist を保存

    graphs.append(graph)
    fit_results.append(fits)

cPeak.Update()
cRes.Update()
cPer.Update()
cHwF.Update()


Removing peak at 1081.1579072362583 because it's too close to 1027.412532326723
切片: -9.64425663366772, 傾き: 5.457472911189342
Removing peak at 1083.42743194333 because it's too close to 1028.2269663395869
Removing peak at 7644.425204576093 because it's too close to 7584.571631633451
切片: -35.02637077620484, 傾き: 5.605370094090969
Peak at 7640.0875 is too close to 7656.6397471953815, skipping.
Removing peak at 947.855154621222 because it's too close to 943.4749586752138
Removing peak at 7483.092350869638 because it's too close to 7427.210733143665
Removing peak at 7588.765748331116 because it's too close to 7582.33579095632
Removing peak at 7659.637314576083 because it's too close to 7656.6397471953815
切片: -29.66578317859899, 傾き: 5.122146263814843
Removing peak at 1086.3597352066377 because it's too close to 1030.993490053453
Removing peak at 7545.490462198161 because it's too close to 7485.922144889402
Removing peak at 7637.9316796105495 because it's too close to 7597.716572104105
Removin

Warning in <TSpectrum::Search>: threshold must 0<threshold<1, threshold=0.05 assumed
Warning in <TSpectrum::Search>: threshold must 0<threshold<1, threshold=0.05 assumed
Warning in <TSpectrum::Search>: threshold must 0<threshold<1, threshold=0.05 assumed
Warning in <TSpectrum::Search>: threshold must 0<threshold<1, threshold=0.05 assumed
Warning in <TSpectrum::Search>: threshold must 0<threshold<1, threshold=0.05 assumed
Warning in <TSpectrum::Search>: threshold must 0<threshold<1, threshold=0.05 assumed
Warning in <TSpectrum::Search>: threshold must 0<threshold<1, threshold=0.05 assumed


In [ ]:
# キャンバスを作成 (6C2 = 15通りなので3x5に分割)
c2D = r.TCanvas("c2D", "2D Histograms", 1200, 800)
c2D.Divide(3, 5)

c2D2 = r.TCanvas("c2D2", "2D Histograms", 1200, 800)
c2D2.Divide(3, 5)

# 組み合わせを保存するリスト
hist2D_list = []

# すべての組み合わせ（6C2 = 15通りの組み合わせ）
comb_list = [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), 
             (1, 2), (1, 3), (1, 4), (1, 5), 
             (2, 3), (2, 4), (2, 5), 
             (3, 4), (3, 5), 
             (4, 5)]

for idx, (i, j) in enumerate(comb_list):
    # 2次元ヒストグラムを描画
    hist_name = f"h2D_{i}_{j}"
    draw_cmd = f"F7IC_Eraw[{i}]:F7IC_Eraw[{j}]>>{hist_name}(2000,0,8000,2000,0,8000)"
    r.tree.Draw(draw_cmd, "", "COLZ")
    
    # キャンバスに描画
    c2D.cd(idx+1)
    r.gPad.SetLogz()
    
    # ヒストグラムを取得
    hist2D = r.gDirectory.Get(hist_name)
    hist2D_list.append(hist2D)
    
    # ヒストグラムを描画
    hist2D.Draw("COLZ")

    c2D2.cd(idx+1)
    hist2D.Draw("COLZ")

# 更新して表示
c2D2.Draw()
